In [10]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [7]:
SEASONS = list(range(2016, 2023))

In [15]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [16]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep*i)

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [27]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

for url in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
        continue

    html = await get_html(url, "#all_schedule")
    with open(save_path, "w+") as f:
        f.write(html)

Timeout error on https://basketball-reference.com/leagues/NBA_2016_games-october.html
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
Timeout error on https://basketball-reference.com/leagues/NBA_2016_games-april.html
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
Timeout error on https://basketball-reference.com/leagues/NBA_2016_games-june.html
2015-16 NBA Schedule | Basketball-Reference.com


In [ ]:
for season in SEASONS:
    await scrape_season(season)

In [33]:
standings_files = os.listdir(STANDINGS_DIR)

In [34]:
standings_files

['NBA_2016_games-april.html',
 'NBA_2016_games-december.html',
 'NBA_2016_games-february.html',
 'NBA_2016_games-january.html',
 'NBA_2016_games-june.html',
 'NBA_2016_games-march.html',
 'NBA_2016_games-may.html',
 'NBA_2016_games-november.html',
 'NBA_2016_games-october.html']

In [35]:
standings_file = os.path.join(STANDINGS_DIR, standings_files[0])
with open(standings_file, 'r') as f:
    html = f.read()

soup = BeautifulSoup(html)
links = soup.find_all("a")
hrefs = [l.get("href") for l in links]
box_scores = [l for l in href if l and "boxscore" in l and ".html" in l]

In [36]:
box_scores

[]